In [ ]:
from tools.project import INPUT_PATH, LOGS_PATH, OUTPUT_PATH, RAW_PATH
import torch
from audiocraft.data.audio import audio_read, audio_write
from audiocraft.data.audio_utils import convert_audio_channels, convert_audio
import numpy as np
from audioldm_eval.metrics.fad import FrechetAudioDistance
import os
import sys
from fadtk.fad import FrechetAudioDistance, log
from fadtk.model_loader import CLAPLaionModel, VGGishModel
from fadtk.fad_batch import cache_embedding_files
from audiocraft.models import MusicGen
import shutil
import contextlib
import io
import warnings
DEVICE = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
EXAMPLES_LEN = 5
torch.cuda.is_available()

In [ ]:
# model = CLAPLaionModel('music')
model = VGGishModel()
eval_dir = OUTPUT_PATH('concepts-dataset', '8bit-slow', 'temp')
cache_embedding_files('fma_pop', model)
cache_embedding_files(eval_dir, model)
fad = FrechetAudioDistance(model, audio_load_worker=8, load_model=False)
fad.score('fma_pop', eval_dir)


In [ ]:
music_model = MusicGen.get_pretrained('facebook/musicgen-small')
music_model.set_generation_params(
	use_sampling=True,
	top_k=250,
	duration=EXAMPLES_LEN
)

In [ ]:
res=music_model.generate([f"music in the style of jazz song"]*5, progress=True)
for a_idx in range(res.shape[0]):
    music = res[a_idx].cpu()
    music = music/np.max(np.abs(music.numpy()))
    path = OUTPUT_PATH("textual-inversion", 'metal', 'temp', f'music_p{a_idx}')
    audio_write(path, music, music_model.cfg.sample_rate)


In [ ]:
cache_embedding_files(OUTPUT_PATH("textual-inversion", 'metal', 'temp'), model)
score = fad.score('fma_pop', OUTPUT_PATH("textual-inversion", 'metal', 'temp'))
shutil.rmtree(os.path.join(OUTPUT_PATH("textual-inversion", 'metal', 'temp'), 'embeddings'))
shutil.rmtree(os.path.join(OUTPUT_PATH("textual-inversion", 'metal', 'temp'), 'convert'))
shutil.rmtree(os.path.join(OUTPUT_PATH("textual-inversion", 'metal', 'temp'), 'stats'))